In [1]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_file, show
import matplotlib.pyplot as plt
from bokeh.models import ColumnDataSource

In [2]:
from bokeh.io import output_notebook, push_notebook
output_notebook()

Loading BokehJS ...

In [3]:
df=pd.read_csv('./crawl_both_reachable.csv',header=None,names=['Website','Http_Time','Https_Time'])
df.head()

,Website,Http_Time,Https_Time
0,google.com,0.0578,0.0960
1,baidu.com,0.5349,1.2672
2,google.co.in,0.0650,0.0956
3,google.co.jp,0.0579,0.0947
4,sohu.com,0.3644,0.4999


In [4]:
def time_diff(r):
    return (r['Https_Time'] -r['Http_Time'])

df2=df.copy()
diff=df2.apply(time_diff,axis=1)
print(np.min(diff),np.max(diff),np.mean(diff))

-7.6158 3.5396 0.13347353735373524


In [5]:
unique=df['Http_Time'].value_counts()
sorted_http=df.sort_values(by=['Http_Time'])
sorted_http.head()

,Website,Http_Time,Https_Time
678,umass.edu,0.0040,0.0190
159,crunchyroll.com,0.0101,0.0328
98,kissanime.ru,0.0110,0.0314
975,scamadviser.com,0.0110,0.0337
841,mangago.me,0.0111,0.1056


In [6]:
sorted_https=df.sort_values(by=['Https_Time'])
sorted_https.tail()

,Website,Http_Time,Https_Time
474,cnnic.cn,2.8031,4.7134
438,wechat.com,5.4636,4.7306
616,piaohua.com,4.4510,4.8884
1064,kanzhun.com,1.4789,5.0185
1046,imperial.ac.uk,5.2499,6.1909


In [20]:
http_times=sorted_http['Http_Time'].value_counts().sort_index()
#print(http_times[:5])
http_times=http_times.cumsum(axis=0)
#print(http_times[:5])
http_times=pd.DataFrame({'time':http_times.index, 'number':http_times.values/max(http_times.values)})

https_times=sorted_https['Https_Time'].value_counts().sort_index()
https_times=https_times.cumsum(axis=0)
https_times=pd.DataFrame({'time':https_times.index, 'number':https_times.values/max(https_times.values)})

In [21]:
https_times.tail()

,number,time
965,0.9964,4.7134
966,0.9973,4.7306
967,0.9982,4.8884
968,0.9991,5.0185
969,1.0000,6.1909


In [22]:

x_range=[min(min(http_times['time']),min(https_times['time'])),\
         max(max(http_times['time']),max(https_times['time']))]
y_range=[min(min(http_times['number']),min(https_times['number'])),max(max(http_times['number'])\
                                                                       ,max(https_times['number']))]
print(x_range,y_range)

plot = figure(title="CDF HTTP vs HTTPS ",y_axis_label="Percent of websites reachable",\
              x_axis_label='Content Download Times(sec)',x_range=x_range,y_range=y_range,\
             width=600,height=300,tools="hover,save")

plot.line(x='time', y='number', legend="HTTP",source=ColumnDataSource(http_times)\
          , line_color="blue", line_width=2)
plot.line(x='time', y='number', legend="HTTPS",source=ColumnDataSource(https_times)\
          , line_color="red", line_width=2)

plot.legend.location='bottom_right'
show(plot)

[0.004, 8.2748] [0.0009000900090009, 1.0]


In [23]:
http_times2=http_times.copy(deep=True)
https_times2=https_times.copy(deep=True)

http_mean=http_times2['time'].mean()
https_mean=https_times2['time'].mean()
http_std=http_times2['time'].std()
https_std=https_times2['time'].std()
print(http_mean,https_mean,http_std,https_std)
th=max(http_mean+2*http_std,https_mean+2*https_std)
print(th)
def drop_outliers(r):
    global th
    return (r['time']<=th)

http_times_=http_times2.apply(drop_outliers,axis=1)
https_times_=https_times2.apply(drop_outliers,axis=1)

http_times2=http_times2.loc[http_times_,:]
https_times2 =https_times2.loc[https_times_,:]

0.37109545454545456 0.5063977319587627 0.6469362373742827 0.70199352220425
1.9103847763672628


In [24]:
x_range=[min(min(http_times2['time']),min(https_times2['time'])),\
         max(max(http_times2['time']),max(https_times2['time']))]
y_range=[min(min(http_times2['number']),min(https_times2['number'])),max(max(http_times2['number'])\
                                                                       ,max(https_times2['number']))]
print(x_range,y_range)

plot = figure(title="CDF HTTP vs HTTPS(subset with t<1.92sec) ",y_axis_label="Percent of websites reachable",\
              x_axis_label='Content Download Times(sec)',x_range=x_range,y_range=y_range,\
             width=600,height=300,tools="hover,save")

plot.line(x='time', y='number', legend="HTTP",source=ColumnDataSource(http_times)\
          , line_color="blue", line_width=2)
plot.line(x='time', y='number', legend="HTTPS",source=ColumnDataSource(https_times)\
          , line_color="red", line_width=2)

plot.legend.location='bottom_right'
show(plot)

[0.004, 1.8788] [0.0009000900090009, 0.9828982898289829]


In [12]:
http_times=sorted_http['Http_Time'].value_counts().sort_index()
#print(http_times[:5])
http_times=http_times.cumsum(axis=0)
#print(http_times[:5])
http_times=pd.DataFrame({'time':http_times.index, 'number':http_times.values})

https_times=sorted_https['Https_Time'].value_counts().sort_index()
https_times=https_times.cumsum(axis=0)
https_times=pd.DataFrame({'time':https_times.index, 'number':https_times.values})

In [13]:

x_range=[min(min(http_times['time']),min(https_times['time'])),\
         max(max(http_times['time']),max(https_times['time']))]
y_range=[min(min(http_times['number']),min(https_times['number'])),max(max(http_times['number'])\
                                                                       ,max(https_times['number']))]
print(x_range,y_range)

plot = figure(title="CDF HTTP vs HTTPS ", x_axis_label='Content Download Times(sec)',x_range=x_range,y_range=y_range,\
             width=600,height=300,tools="hover,save",y_axis_label='Number of requests')

plot.line(x='time', y='number', legend="HTTP",source=ColumnDataSource(http_times)\
          , line_color="blue", line_width=2)
plot.line(x='time', y='number', legend="HTTPS",source=ColumnDataSource(https_times)\
          , line_color="red", line_width=2)

plot.legend.location='bottom_right'
show(plot)

[0.004, 8.2748] [1, 1111]
